In [1]:
import pandas as pd

In [2]:
pd.set_option("max_columns",100)

In [70]:
tweets = pd.read_pickle("../../data/cp_tweets_classified.zip")

## have a lok at the data and apply preproccessing to it

In [50]:
import sys
lib_dir = "../../libs/"
if lib_dir not in sys.path:
    sys.path.append(lib_dir)

In [30]:
import preprocessor

## Stemming & Lammatization

In [64]:
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import * 

def stem(df):

    stemmer = SnowballStemmer("english")
    
    text_stem_lam = df.sample(100000).text.apply(lambda x: x.split(' '))\
                                        .apply(lambda l:
    list(map(
        lambda x: WordNetLemmatizer().lemmatize(x, pos='v'),
        l))
    )

    return text_stem_lam

In [84]:
def get_word_dictionary(text):
    
    # Importing Gensim
    import gensim
    from gensim import corpora

    # Creating the term dictionary of our courpus, where every unique term is assigned an index
    dictionary = corpora.Dictionary(text)

    # Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
    doc_term_matrix = [dictionary.doc2bow(doc) for doc in text]
    
    return dictionary,doc_term_matrix

## Word Dictionary

In [85]:
#x = tweets.sample(100000).clean_text.apply(lambda x: x.split(' '))
#doc_clean = list(x)
doc_clean = stem(tweets[tweets.about == 1]) #about Hillary
dictionary,doc_term_matrix = get_word_dictionary(doc_clean)

2018-10-03 13:42:00,984 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-10-03 13:42:02,318 : INFO : adding document #10000 to Dictionary(33807 unique tokens: ['@HillaryClinton', '@JenGranholm', '@MelissaJaneSays', '@Ricky_Vaughn99', 'a']...)
2018-10-03 13:42:03,346 : INFO : adding document #20000 to Dictionary(56740 unique tokens: ['@HillaryClinton', '@JenGranholm', '@MelissaJaneSays', '@Ricky_Vaughn99', 'a']...)
2018-10-03 13:42:04,547 : INFO : adding document #30000 to Dictionary(76933 unique tokens: ['@HillaryClinton', '@JenGranholm', '@MelissaJaneSays', '@Ricky_Vaughn99', 'a']...)
2018-10-03 13:42:05,856 : INFO : adding document #40000 to Dictionary(95571 unique tokens: ['@HillaryClinton', '@JenGranholm', '@MelissaJaneSays', '@Ricky_Vaughn99', 'a']...)
2018-10-03 13:42:06,968 : INFO : adding document #50000 to Dictionary(112583 unique tokens: ['@HillaryClinton', '@JenGranholm', '@MelissaJaneSays', '@Ricky_Vaughn99', 'a']...)
2018-10-03 13:42:08,382 : INFO : addi

## LDA Model

In [86]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

**NOTE** the higher the `passes` arg is the more time it will take to train. Im not sure if it guarentees better topics.

In [87]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=10, id2word = dictionary, passes=5)

2018-10-03 13:43:48,021 : INFO : using symmetric alpha at 0.1
2018-10-03 13:43:48,023 : INFO : using symmetric eta at 0.1
2018-10-03 13:43:48,106 : INFO : using serial LDA version on this node
2018-10-03 13:43:48,535 : INFO : running online (multi-pass) LDA training, 10 topics, 5 passes over the supplied corpus of 100000 documents, updating model once every 2000 documents, evaluating perplexity every 20000 documents, iterating 50x with a convergence threshold of 0.001000
2018-10-03 13:43:48,541 : INFO : PROGRESS: pass 0, at document #2000/100000
2018-10-03 13:43:52,752 : INFO : merging changes from 2000 documents into a model of 100000 documents
2018-10-03 13:43:53,616 : INFO : topic #6 (0.100): 0.031*"@HillaryClinton" + 0.020*"be" + 0.015*"a" + 0.010*"of" + 0.009*"do" + 0.009*"have" + 0.009*"" + 0.009*"the" + 0.009*"to" + 0.009*"and"
2018-10-03 13:43:53,626 : INFO : topic #9 (0.100): 0.054*"@HillaryClinton" + 0.028*"be" + 0.016*"the" + 0.016*"" + 0.014*"of" + 0.011*"to" + 0.009*"a" + 

2018-10-03 13:44:13,836 : INFO : topic #6 (0.100): 0.024*"@HillaryClinton" + 0.011*"be" + 0.011*"a" + 0.010*"" + 0.010*"#MAGA" + 0.007*"of" + 0.007*"have" + 0.007*"#Hillary" + 0.007*"her" + 0.006*"the"
2018-10-03 13:44:13,849 : INFO : topic #8 (0.100): 0.029*"@HillaryClinton" + 0.028*"be" + 0.017*"a" + 0.017*"to" + 0.017*"you" + 0.016*"have" + 0.015*"I" + 0.014*"for" + 0.010*"the" + 0.010*"do"
2018-10-03 13:44:13,868 : INFO : topic #0 (0.100): 0.049*"be" + 0.040*"the" + 0.038*"@HillaryClinton" + 0.025*"to" + 0.016*"a" + 0.016*"of" + 0.015*"for" + 0.012*"and" + 0.010*"in" + 0.009*"on"
2018-10-03 13:44:13,878 : INFO : topic #5 (0.100): 0.048*"@HillaryClinton" + 0.033*"" + 0.022*"be" + 0.015*"the" + 0.012*"of" + 0.010*"a" + 0.008*"to" + 0.008*"for" + 0.008*"and" + 0.007*"you"
2018-10-03 13:44:13,890 : INFO : topic diff=0.154039, rho=0.377964
2018-10-03 13:44:13,900 : INFO : PROGRESS: pass 0, at document #16000/100000
2018-10-03 13:44:16,173 : INFO : merging changes from 2000 documents int

2018-10-03 13:44:36,362 : INFO : topic #0 (0.100): 0.050*"be" + 0.043*"the" + 0.038*"@HillaryClinton" + 0.024*"to" + 0.021*"of" + 0.017*"a" + 0.016*"for" + 0.012*"and" + 0.012*"in" + 0.008*"on"
2018-10-03 13:44:36,370 : INFO : topic #4 (0.100): 0.037*"#HillaryClinton" + 0.008*"#Hillary" + 0.008*"#ImWithHer" + 0.006*"via" + 0.006*"#tcot" + 0.005*"@HillaryClinton" + 0.005*"" + 0.004*"The" + 0.004*"#imwithher" + 0.003*"#CrookedHillary"
2018-10-03 13:44:36,379 : INFO : topic diff=0.152970, rho=0.277350
2018-10-03 13:44:36,385 : INFO : PROGRESS: pass 0, at document #28000/100000
2018-10-03 13:44:38,840 : INFO : merging changes from 2000 documents into a model of 100000 documents
2018-10-03 13:44:39,378 : INFO : topic #7 (0.100): 0.101*"" + 0.035*"@HillaryClinton" + 0.021*"#ImWithHer" + 0.021*"#NeverHillary" + 0.005*"#StrongerTogether" + 0.005*"@timkaine" + 0.005*"a" + 0.004*"of" + 0.004*"In" + 0.004*"#hillary"
2018-10-03 13:44:39,386 : INFO : topic #0 (0.100): 0.050*"be" + 0.044*"the" + 0.0

2018-10-03 13:44:56,960 : INFO : topic diff=0.156381, rho=0.229416
2018-10-03 13:45:00,955 : INFO : -10.210 per-word bound, 1184.4 perplexity estimate based on a held-out corpus of 2000 documents with 28556 words
2018-10-03 13:45:00,957 : INFO : PROGRESS: pass 0, at document #40000/100000
2018-10-03 13:45:03,352 : INFO : merging changes from 2000 documents into a model of 100000 documents
2018-10-03 13:45:03,764 : INFO : topic #5 (0.100): 0.046*"@HillaryClinton" + 0.030*"" + 0.013*"@CNN" + 0.011*"@HFA" + 0.010*"be" + 0.008*"@MSNBC" + 0.007*"#ImWithHer" + 0.007*"of" + 0.006*"the" + 0.006*"a"
2018-10-03 13:45:03,771 : INFO : topic #8 (0.100): 0.034*"be" + 0.031*"@HillaryClinton" + 0.019*"a" + 0.019*"I" + 0.016*"to" + 0.014*"for" + 0.014*"have" + 0.012*"you" + 0.011*"that" + 0.010*"the"
2018-10-03 13:45:03,774 : INFO : topic #9 (0.100): 0.047*"@HillaryClinton" + 0.010*"A" + 0.010*"U" + 0.010*"THE" + 0.009*"&amp;" + 0.008*"" + 0.008*"YOU" + 0.006*"TO" + 0.006*"IS" + 0.006*"2"
2018-10-03 13

2018-10-03 13:45:17,886 : INFO : PROGRESS: pass 0, at document #52000/100000
2018-10-03 13:45:20,373 : INFO : merging changes from 2000 documents into a model of 100000 documents
2018-10-03 13:45:20,795 : INFO : topic #5 (0.100): 0.045*"@HillaryClinton" + 0.033*"" + 0.015*"@CNN" + 0.013*"@HFA" + 0.008*"@MSNBC" + 0.007*"be" + 0.007*"#ImWithHer" + 0.006*"@RogerJStoneJr" + 0.006*"@OnMessageForHer" + 0.006*"@EricSuper5"
2018-10-03 13:45:20,800 : INFO : topic #2 (0.100): 0.054*"@HillaryClinton" + 0.034*"to" + 0.026*"be" + 0.025*"you" + 0.021*"the" + 0.020*"a" + 0.016*"and" + 0.013*"I" + 0.009*"have" + 0.009*"your"
2018-10-03 13:45:20,804 : INFO : topic #6 (0.100): 0.016*"#MAGA" + 0.012*"" + 0.012*"#NeverHillary" + 0.009*"@HillaryClinton" + 0.007*"#Hillary" + 0.007*"#NEVERHILLARY" + 0.005*"#HEXIT" + 0.005*"#VOTEGOP" + 0.005*"#DemExit" + 0.004*"#StopHillary"
2018-10-03 13:45:20,811 : INFO : topic #0 (0.100): 0.049*"be" + 0.043*"the" + 0.039*"@HillaryClinton" + 0.025*"to" + 0.023*"of" + 0.019*

...

2018-10-03 13:54:47,301 : INFO : topic #9 (0.100): 0.042*"@HillaryClinton" + 0.014*"A" + 0.011*"&amp;" + 0.009*"U" + 0.009*"THE" + 0.009*"YOU" + 0.007*"IS" + 0.007*"TO" + 0.006*"#TrumpPence16" + 0.005*""
2018-10-03 13:54:47,305 : INFO : topic #1 (0.100): 0.119*"@HillaryClinton" + 0.010*"@FoxNews" + 0.010*"@BarackObama" + 0.004*"@POTUS" + 0.004*"HER" + 0.004*"@seanhannity" + 0.003*"@JudgeJeanine" + 0.003*"@politico" + 0.003*"@Reince" + 0.003*"@JoeBiden"
2018-10-03 13:54:47,308 : INFO : topic #5 (0.100): 0.043*"@HillaryClinton" + 0.031*"" + 0.024*"@CNN" + 0.015*"@HFA" + 0.013*"@MSNBC" + 0.007*"#ImWithHer" + 0.007*"@OnMessageForHer" + 0.007*"@RogerJStoneJr" + 0.006*"@EricSuper5" + 0.006*"@realkingrobbo"
2018-10-03 13:54:47,312 : INFO : topic #7 (0.100): 0.125*"" + 0.034*"@HillaryClinton" + 0.031*"#NeverHillary" + 0.026*"#ImWithHer" + 0.006*"#StrongerTogether" + 0.006*"@timkaine" + 0.005*"@billclinton" + 0.005*"In" + 0.004*"#NeverTrump" + 0.003*"#LockHerUp"
2018-10-03 13:54:47,329 : INFO :

2018-10-03 13:54:56,619 : INFO : topic #8 (0.100): 0.022*"be" + 0.020*"@HillaryClinton" + 0.019*"a" + 0.018*"I" + 0.011*"for" + 0.009*"#ImWithHer" + 0.008*"#Hillary" + 0.007*"have" + 0.006*"on" + 0.006*"I'm"
2018-10-03 13:54:56,623 : INFO : topic #6 (0.100): 0.019*"#MAGA" + 0.015*"#NeverHillary" + 0.012*"" + 0.008*"#NEVERHILLARY" + 0.006*"#HEXIT" + 0.005*"#StopHillary" + 0.005*"#Hillary" + 0.005*"Trump." + 0.005*"#Benghazi" + 0.005*"#DemExit"
2018-10-03 13:54:56,626 : INFO : topic #0 (0.100): 0.048*"be" + 0.044*"the" + 0.038*"@HillaryClinton" + 0.026*"of" + 0.025*"to" + 0.021*"a" + 0.015*"for" + 0.015*"in" + 0.014*"and" + 0.010*"on"
2018-10-03 13:54:56,632 : INFO : topic #4 (0.100): 0.034*"#HillaryClinton" + 0.010*"#Hillary" + 0.006*"via" + 0.006*"#tcot" + 0.004*"#imwithher" + 0.004*"The" + 0.004*"@" + 0.004*"Her" + 0.004*"State" + 0.004*"#ImWithHer"
2018-10-03 13:54:56,651 : INFO : topic diff=0.121378, rho=0.134840
2018-10-03 13:54:56,658 : INFO : PROGRESS: pass 4, at document #64000/

2018-10-03 13:55:01,246 : INFO : topic #1 (0.100): 0.119*"@HillaryClinton" + 0.010*"@FoxNews" + 0.009*"@BarackObama" + 0.005*"@POTUS" + 0.004*"@JoeBiden" + 0.003*"@seanhannity" + 0.003*"@Reince" + 0.003*"@AP" + 0.003*"HER" + 0.003*"@JudgeJeanine"
2018-10-03 13:55:01,251 : INFO : topic #3 (0.100): 0.030*"@HillaryClinton" + 0.007*"de" + 0.006*"To" + 0.005*"never" + 0.005*"la" + 0.004*"ever" + 0.004*"a" + 0.004*"#MAGA" + 0.004*"@CNNPolitics" + 0.004*"Vote"
2018-10-03 13:55:01,255 : INFO : topic #9 (0.100): 0.040*"@HillaryClinton" + 0.016*"A" + 0.011*"&amp;" + 0.010*"THE" + 0.009*"YOU" + 0.008*"U" + 0.008*"TO" + 0.007*"IS" + 0.006*"NOT" + 0.006*"#TrumpPence16"
2018-10-03 13:55:01,271 : INFO : topic diff=0.126893, rho=0.134840
2018-10-03 13:55:01,278 : INFO : PROGRESS: pass 4, at document #76000/100000
2018-10-03 13:55:01,863 : INFO : merging changes from 2000 documents into a model of 100000 documents
2018-10-03 13:55:02,025 : INFO : topic #5 (0.100): 0.042*"@HillaryClinton" + 0.031*"" + 0

2018-10-03 13:55:08,596 : INFO : topic #6 (0.100): 0.018*"#MAGA" + 0.014*"#NeverHillary" + 0.012*"" + 0.007*"#NEVERHILLARY" + 0.006*"#HEXIT" + 0.005*"#VOTEGOP" + 0.005*"Trump." + 0.005*"#StopHillary" + 0.005*"#CrookedHillary" + 0.005*"#Hillary"
2018-10-03 13:55:08,603 : INFO : topic #7 (0.100): 0.121*"" + 0.035*"@HillaryClinton" + 0.028*"#NeverHillary" + 0.028*"#ImWithHer" + 0.005*"@billclinton" + 0.005*"@timkaine" + 0.005*"#StrongerTogether" + 0.005*"In" + 0.004*"#hillary" + 0.003*"#NeverTrump"
2018-10-03 13:55:08,608 : INFO : topic #5 (0.100): 0.041*"@HillaryClinton" + 0.031*"" + 0.024*"@CNN" + 0.014*"@HFA" + 0.013*"@MSNBC" + 0.008*"#ImWithHer" + 0.006*"@OnMessageForHer" + 0.006*"@RogerJStoneJr" + 0.006*"@realkingrobbo" + 0.005*"@EricSuper5"
2018-10-03 13:55:08,618 : INFO : topic diff=0.126070, rho=0.134840
2018-10-03 13:55:08,626 : INFO : PROGRESS: pass 4, at document #88000/100000
2018-10-03 13:55:09,693 : INFO : merging changes from 2000 documents into a model of 100000 documents


2018-10-03 13:55:18,523 : INFO : topic #0 (0.100): 0.049*"be" + 0.043*"the" + 0.038*"@HillaryClinton" + 0.025*"of" + 0.025*"to" + 0.022*"a" + 0.015*"for" + 0.015*"in" + 0.014*"and" + 0.010*"&amp;"
2018-10-03 13:55:18,529 : INFO : topic #7 (0.100): 0.118*"" + 0.034*"@HillaryClinton" + 0.029*"#NeverHillary" + 0.028*"#ImWithHer" + 0.006*"#StrongerTogether" + 0.006*"@timkaine" + 0.005*"In" + 0.005*"@billclinton" + 0.003*"#hillary" + 0.003*"Great"
2018-10-03 13:55:18,536 : INFO : topic #6 (0.100): 0.019*"#MAGA" + 0.015*"#NeverHillary" + 0.012*"" + 0.007*"#NEVERHILLARY" + 0.006*"#HEXIT" + 0.005*"Trump." + 0.005*"#VOTEGOP" + 0.005*"#CrookedHillary" + 0.005*"#Hillary" + 0.004*"#StopHillary"
2018-10-03 13:55:18,545 : INFO : topic diff=0.124430, rho=0.134840
2018-10-03 13:55:21,177 : INFO : -9.904 per-word bound, 958.0 perplexity estimate based on a held-out corpus of 2000 documents with 28443 words
2018-10-03 13:55:21,180 : INFO : PROGRESS: pass 4, at document #100000/100000
2018-10-03 13:55:22

In [89]:
# Print 2 topics and describe then with 4 words.
topics = ldamodel.print_topics(num_topics=10, num_words=10)

i=0
for topic in topics:
    print ("Topic",i ,"->", topic)     
    i+=1


2018-10-03 13:57:56,220 : INFO : topic #0 (0.100): 0.049*"be" + 0.043*"the" + 0.037*"@HillaryClinton" + 0.025*"of" + 0.025*"to" + 0.022*"a" + 0.015*"for" + 0.015*"in" + 0.014*"and" + 0.010*"&amp;"
2018-10-03 13:57:56,225 : INFO : topic #1 (0.100): 0.123*"@HillaryClinton" + 0.012*"@FoxNews" + 0.007*"@BarackObama" + 0.005*"@seanhannity" + 0.004*"@POTUS" + 0.004*"@Reince" + 0.004*"@politico" + 0.003*"@AP" + 0.003*"HER" + 0.003*"Another"
2018-10-03 13:57:56,228 : INFO : topic #2 (0.100): 0.057*"@HillaryClinton" + 0.034*"be" + 0.031*"to" + 0.030*"you" + 0.022*"the" + 0.017*"a" + 0.016*"I" + 0.015*"and" + 0.012*"have" + 0.011*"do"
2018-10-03 13:57:56,232 : INFO : topic #3 (0.100): 0.029*"@HillaryClinton" + 0.006*"To" + 0.006*"de" + 0.006*"never" + 0.005*"ever" + 0.005*"#MAGA" + 0.005*"Vote" + 0.004*"a" + 0.003*"Of" + 0.003*"@CNNPolitics"
2018-10-03 13:57:56,236 : INFO : topic #4 (0.100): 0.034*"#HillaryClinton" + 0.008*"#Hillary" + 0.006*"via" + 0.005*"#imwithher" + 0.005*"#tcot" + 0.004*"He

Topic 0 -> (0, '0.049*"be" + 0.043*"the" + 0.037*"@HillaryClinton" + 0.025*"of" + 0.025*"to" + 0.022*"a" + 0.015*"for" + 0.015*"in" + 0.014*"and" + 0.010*"&amp;"')
Topic 1 -> (1, '0.123*"@HillaryClinton" + 0.012*"@FoxNews" + 0.007*"@BarackObama" + 0.005*"@seanhannity" + 0.004*"@POTUS" + 0.004*"@Reince" + 0.004*"@politico" + 0.003*"@AP" + 0.003*"HER" + 0.003*"Another"')
Topic 2 -> (2, '0.057*"@HillaryClinton" + 0.034*"be" + 0.031*"to" + 0.030*"you" + 0.022*"the" + 0.017*"a" + 0.016*"I" + 0.015*"and" + 0.012*"have" + 0.011*"do"')
Topic 3 -> (3, '0.029*"@HillaryClinton" + 0.006*"To" + 0.006*"de" + 0.006*"never" + 0.005*"ever" + 0.005*"#MAGA" + 0.005*"Vote" + 0.004*"a" + 0.003*"Of" + 0.003*"@CNNPolitics"')
Topic 4 -> (4, '0.034*"#HillaryClinton" + 0.008*"#Hillary" + 0.006*"via" + 0.005*"#imwithher" + 0.005*"#tcot" + 0.004*"Her" + 0.004*"@" + 0.004*"State" + 0.004*"The" + 0.003*"#Clinton"')
Topic 5 -> (5, '0.042*"@HillaryClinton" + 0.030*"" + 0.024*"@CNN" + 0.014*"@HFA" + 0.012*"@MSNBC" + 0

In [81]:
tweets[tweets.about == 0].shape

(363284, 34)

In [75]:
#x = tweets.sample(100000).clean_text.apply(lambda x: x.split(' '))
#doc_clean = list(x)
doc_clean = stem(tweets[tweets.about == 0]) #about Trump
doc_term_matrix = get_word_dictionary(doc_clean)

2018-10-03 13:31:10,951 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-10-03 13:31:11,777 : INFO : adding document #10000 to Dictionary(34087 unique tokens: ['@FoxNews', '@jpodhoretz', '@realDonaldTrump', '@seanhannity', 'belief.']...)
2018-10-03 13:31:12,652 : INFO : adding document #20000 to Dictionary(57666 unique tokens: ['@FoxNews', '@jpodhoretz', '@realDonaldTrump', '@seanhannity', 'belief.']...)
2018-10-03 13:31:13,548 : INFO : adding document #30000 to Dictionary(78278 unique tokens: ['@FoxNews', '@jpodhoretz', '@realDonaldTrump', '@seanhannity', 'belief.']...)
2018-10-03 13:31:14,364 : INFO : adding document #40000 to Dictionary(96871 unique tokens: ['@FoxNews', '@jpodhoretz', '@realDonaldTrump', '@seanhannity', 'belief.']...)
2018-10-03 13:31:15,119 : INFO : adding document #50000 to Dictionary(114626 unique tokens: ['@FoxNews', '@jpodhoretz', '@realDonaldTrump', '@seanhannity', 'belief.']...)
2018-10-03 13:31:15,819 : INFO : adding document #60000 to Dic

In [76]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=10, id2word = dictionary, passes=5)

2018-10-03 13:35:11,797 : INFO : using symmetric alpha at 0.1
2018-10-03 13:35:11,800 : INFO : using symmetric eta at 0.1
2018-10-03 13:35:11,827 : INFO : using serial LDA version on this node
2018-10-03 13:35:12,072 : INFO : running online (multi-pass) LDA training, 10 topics, 5 passes over the supplied corpus of 100000 documents, updating model once every 2000 documents, evaluating perplexity every 20000 documents, iterating 50x with a convergence threshold of 0.001000
2018-10-03 13:35:12,126 : INFO : PROGRESS: pass 0, at document #2000/100000
2018-10-03 13:35:16,933 : INFO : merging changes from 2000 documents into a model of 100000 documents
2018-10-03 13:35:17,152 : INFO : topic #1 (0.100): 0.041*"#hillari" + 0.025*"idol" + 0.024*"say" + 0.021*"accept" + 0.016*"repeat" + 0.015*"want" + 0.012*"long" + 0.012*"via" + 0.010*"hear" + 0.009*"mexico"
2018-10-03 13:35:17,158 : INFO : topic #9 (0.100): 0.044*"#hillari" + 0.025*"accept" + 0.025*"say" + 0.022*"#proud" + 0.016*"repeat" + 0.01

2018-10-03 13:35:37,555 : INFO : topic diff=0.359446, rho=0.408248
2018-10-03 13:35:37,584 : INFO : PROGRESS: pass 0, at document #14000/100000
2018-10-03 13:35:40,638 : INFO : merging changes from 2000 documents into a model of 100000 documents
2018-10-03 13:35:40,798 : INFO : topic #5 (0.100): 0.077*"hear" + 0.045*"say" + 0.038*"#hillari" + 0.017*"accept" + 0.012*"idol" + 0.011*"repeat" + 0.011*"sound" + 0.011*"#donaldtrump" + 0.008*"#proud" + 0.008*"#rudygiuliani"
2018-10-03 13:35:40,800 : INFO : topic #8 (0.100): 0.044*"#hillari" + 0.034*"say" + 0.031*"repeat" + 0.023*"idol" + 0.021*"via" + 0.015*"accept" + 0.012*"hear" + 0.011*"sound" + 0.011*"mexico" + 0.010*"#donaldtrump"
2018-10-03 13:35:40,803 : INFO : topic #7 (0.100): 0.042*"#hillari" + 0.022*"polici" + 0.014*"#donaldtrump#mexicosharedprincipl" + 0.014*"#donaldtrump" + 0.012*"accept" + 0.010*"say" + 0.010*"hear" + 0.010*"show" + 0.009*"via" + 0.009*"we"
2018-10-03 13:35:40,805 : INFO : topic #6 (0.100): 0.056*"#hillari" + 0.

2018-10-03 13:36:02,770 : INFO : topic diff=0.372301, rho=0.288675
2018-10-03 13:36:02,793 : INFO : PROGRESS: pass 0, at document #26000/100000
2018-10-03 13:36:06,143 : INFO : merging changes from 2000 documents into a model of 100000 documents
2018-10-03 13:36:06,373 : INFO : topic #9 (0.100): 0.045*"#hillari" + 0.042*"accept" + 0.038*"say" + 0.027*"repeat" + 0.022*"#proud" + 0.016*"via" + 0.016*"long" + 0.014*"show" + 0.010*"flack" + 0.010*"#donaldtrump"
2018-10-03 13:36:06,376 : INFO : topic #3 (0.100): 0.124*"#hillari" + 0.029*"hear" + 0.015*"#crookedhillari" + 0.010*"say" + 0.010*"#imwithhercolosseum" + 0.008*"#fellowdeplor" + 0.007*"background" + 0.007*"accept" + 0.006*"long" + 0.004*"mediocraci"
2018-10-03 13:36:06,379 : INFO : topic #8 (0.100): 0.046*"#hillari" + 0.036*"say" + 0.030*"repeat" + 0.023*"idol" + 0.020*"via" + 0.016*"accept" + 0.011*"sound" + 0.010*"mexico" + 0.010*"#donaldtrump" + 0.009*"elect"
2018-10-03 13:36:06,383 : INFO : topic #7 (0.100): 0.040*"#hillari" + 

IndexError: index 73565 is out of bounds for axis 1 with size 73565

In [77]:
# Print 2 topics and describe then with 4 words.
topics = ldamodel.print_topics(num_topics=10, num_words=6)

i=0
for topic in topics:
    print ("Topic",i ,"->", topic)     
    i+=1



2018-10-03 13:38:07,769 : INFO : topic #0 (0.100): 0.018*"let" + 0.014*"a" + 0.012*"care" + 0.011*"dumb" + 0.011*"junto" + 0.010*"all"
2018-10-03 13:38:07,772 : INFO : topic #1 (0.100): 0.040*"donni" + 0.016*"shove" + 0.015*"moment" + 0.014*"andor" + 0.012*"feel" + 0.011*"appreci"
2018-10-03 13:38:07,774 : INFO : topic #2 (0.100): 0.022*"#newmexicotru" + 0.013*"deliv" + 0.013*"scotland" + 0.012*"shut" + 0.012*"yesterday" + 0.012*"esperar"
2018-10-03 13:38:07,777 : INFO : topic #3 (0.100): 0.042*"fake" + 0.024*"girl" + 0.015*"hour" + 0.015*"actual" + 0.013*"possibl" + 0.013*"alli"
2018-10-03 13:38:07,779 : INFO : topic #4 (0.100): 0.014*"#gopvet" + 0.014*"benefit" + 0.013*"instead" + 0.011*"plight" + 0.010*"yes" + 0.009*"concentr"
2018-10-03 13:38:07,781 : INFO : topic #5 (0.100): 0.033*"hes" + 0.021*"pickl" + 0.014*"#donaldtrump#dncleak" + 0.013*"peachi" + 0.012*"name" + 0.012*"fake"
2018-10-03 13:38:07,784 : INFO : topic #6 (0.100): 0.018*"still" + 0.017*"o" + 0.016*"man" + 0.016*"#ms

Topic 0 -> (0, '0.018*"let" + 0.014*"a" + 0.012*"care" + 0.011*"dumb" + 0.011*"junto" + 0.010*"all"')
Topic 1 -> (1, '0.040*"donni" + 0.016*"shove" + 0.015*"moment" + 0.014*"andor" + 0.012*"feel" + 0.011*"appreci"')
Topic 2 -> (2, '0.022*"#newmexicotru" + 0.013*"deliv" + 0.013*"scotland" + 0.012*"shut" + 0.012*"yesterday" + 0.012*"esperar"')
Topic 3 -> (3, '0.042*"fake" + 0.024*"girl" + 0.015*"hour" + 0.015*"actual" + 0.013*"possibl" + 0.013*"alli"')
Topic 4 -> (4, '0.014*"#gopvet" + 0.014*"benefit" + 0.013*"instead" + 0.011*"plight" + 0.010*"yes" + 0.009*"concentr"')
Topic 5 -> (5, '0.033*"hes" + 0.021*"pickl" + 0.014*"#donaldtrump#dncleak" + 0.013*"peachi" + 0.012*"name" + 0.012*"fake"')
Topic 6 -> (6, '0.018*"still" + 0.017*"o" + 0.016*"man" + 0.016*"#msm" + 0.012*"sthu" + 0.012*"naacp"')
Topic 7 -> (7, '0.023*"para" + 0.019*"didnt" + 0.019*"him" + 0.017*"power" + 0.012*"final" + 0.008*"nose"')
Topic 8 -> (8, '0.019*"we" + 0.012*"economi" + 0.012*"dirigir" + 0.011*"#biden" + 0.010*"